# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 

from matplotlib import pyplot as plt
from matplotlib import lines
from matplotlib.patches import Rectangle

import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '..', 'pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils
import plot_utils

plot_utils.set_rcParams()

In [ ]:
fig_num1 = os.getcwd().split('/')[-1][3:5]
print(fig_num1)

fig_num2 = os.getcwd().split('/')[-1][6:8]
print(fig_num2)

# Get data

In [ ]:
cone_optim_folder = os.path.join('..', '..', 'step1a_optimize_cones', 'optim_data')
cbc_optim_folder = os.path.join('..', '..', 'step2a_optimize_cbc', 'optim_data')

In [ ]:
os.listdir(cone_optim_folder)

In [ ]:
sorted(os.listdir(cbc_optim_folder))

In [ ]:
cell2folder = {
    'Cone': os.path.join(cone_optim_folder, 'optimize_cone_submission2'),
    'OFF':  os.path.join(cbc_optim_folder, 'optimize_OFF_submission2'),
    'ON':   os.path.join(cbc_optim_folder, 'optimize_ON_submission2'),
}

In [ ]:
cell2data = {}
for cell, folder in cell2folder.items():
    cell2data[cell] = {}
    
    rec_time = data_utils.load_var(os.path.join(folder, 'post_data', 'rec_time.pkl')) 
    idxs = np.arange(0,rec_time.size,10)
    
    cell2data[cell]['rec_time'] = rec_time[idxs]
    cell2data[cell]['iGlus']    = data_utils.load_var(os.path.join(folder, 'post_data', 'iGlus.pkl'))
    cell2data[cell]['Vms']      = data_utils.load_var(os.path.join(folder, 'post_data', 'Vms.pkl'))[:,idxs]
    cell2data[cell]['rates']    = data_utils.load_var(os.path.join(folder, 'post_data', 'rates.pkl'))[:,idxs]
    
    final_model_output = data_utils.load_var(os.path.join(folder, 'post_data', 'final_model_output.pkl'))
    
    cell2data[cell]['iGlu_best']   = final_model_output['iGlu']
    cell2data[cell]['Vm_best']     = final_model_output['Vm'][idxs]
    cell2data[cell]['rate_best']   = final_model_output['rate'][idxs]
    
    cell2data[cell]['Stim']        = final_model_output['Stimulus']
    
    cell2data[cell]['Target']      = final_model_output['Target']
    cell2data[cell]['Time-Target'] = final_model_output['Time-Target']

# Save source data

In [ ]:
data_utils.make_dir('source_data/Cone')
data_utils.make_dir('source_data/CBC')

## Export stimuli

In [ ]:
cell2data['Cone']['Stim'].to_csv(
    'source_data/Cone/Stimulus.csv',
    float_format="%.4f", header=['Time/s', 'Normalized Stimulus'], index=False
)

pd.read_csv('source_data/Cone/Stimulus.csv').plot(x='Time/s', figsize=(8,1));

In [ ]:
cell2data['ON']['Stim'].to_csv(
    'source_data/CBC/Stimulus.csv',
    float_format="%.4f", header=['Time/s', 'Normalized Stimulus'], index=False
)

pd.read_csv('source_data/CBC/Stimulus.csv').plot(x='Time/s', figsize=(8,1));

## iGluSnFR

In [ ]:
for cell in cell2folder.keys():
    
    if cell == 'Cone':
        filename= 'source_data/Cone/iGluSnFR_target_and_samples.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_iGluSnFR_target_and_samples.csv'
        
    iGlu_cols = ['Sample' + str(i+1) + ': iGluSnFR' for i in range(200)]
    
    iGluSnFR_exdf = pd.DataFrame(columns=['Time/s', 'Target: iGluSnFR', 'Best Sample: iGluSnFR'] + iGlu_cols)
        
    iGluSnFR_exdf['Time/s'] = cell2data[cell]['Time-Target']
    iGluSnFR_exdf['Target: iGluSnFR'] = cell2data[cell]['Target']
    iGluSnFR_exdf['Best Sample: iGluSnFR'] = cell2data[cell]['iGlu_best']
   
    iGluSnFR_exdf[iGlu_cols] = cell2data[cell]['iGlus'].T
    
    iGluSnFR_exdf.to_csv(filename, float_format="%.4f", index=False) 

In [ ]:
for cell in cell2folder.keys():
    
    if cell == 'Cone':
        filename= 'source_data/Cone/iGluSnFR_target_and_samples.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_iGluSnFR_target_and_samples.csv'

    fig, ax = plt.subplots(1,1,figsize=(8,2))
    iGluSnFR_exdf = pd.read_csv(filename)
    iGluSnFR_exdf.plot(x='Time/s', legend=None, ax=ax, color='gray', lw=0.5)
    ax.plot(iGluSnFR_exdf['Time/s'], iGluSnFR_exdf['Target: iGluSnFR'], lw=2)
    ax.plot(iGluSnFR_exdf['Time/s'], iGluSnFR_exdf['Best Sample: iGluSnFR'], lw=1)

## Rate and Vms

In [ ]:
for cell in cell2folder.keys():
    
    if cell == 'Cone':
        filename= 'source_data/Cone/rates_and_Vms_target_and_samples.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_rates_and_Vms_target_and_samples.csv'
        
    rate_cols = ['Sample' + str(i+1) + ': Mean release rate/(ves./s)' for i in range(200)]
    Vm_cols = ['Sample' + str(i+1) + ': Somatic membrane potential/mV' for i in range(200)]
    
    ratesVms_exdf = pd.DataFrame(
        columns=['Time/s', 'Target: iGluSnFR'] +\
        ['Best Sample: Mean release rate/(ves./s)' + 'Best Sample: Somatic membrane potential/mV'] +\
        rate_cols + Vm_cols)
        
    ratesVms_exdf['Time/s'] = cell2data[cell]['rec_time']
   
    ratesVms_exdf['Best Sample: Mean release rate/(ves./s)'] = cell2data[cell]['rate_best']
    ratesVms_exdf['Best Sample: Somatic membrane potential/mV'] = cell2data[cell]['Vm_best']*1e3

    ratesVms_exdf[rate_cols] = cell2data[cell]['rates'].T
    ratesVms_exdf[Vm_cols] = cell2data[cell]['Vms'].T*1e3
    
    ratesVms_exdf.to_csv(filename, float_format="%.4f", index=False) 

In [ ]:
for cell in cell2folder.keys():
    
    if cell == 'Cone':
        filename= 'source_data/Cone/rates_and_Vms_target_and_samples.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_rates_and_Vms_target_and_samples.csv'

    rate_cols = ['Sample' + str(i+1) + ': Mean release rate/(ves./s)' for i in range(200)]
    Vm_cols = ['Sample' + str(i+1) + ': Somatic membrane potential/mV' for i in range(200)]
        
    fig, axs = plt.subplots(1,2,figsize=(8,2))
    ratesVms_exdf = pd.read_csv(filename)
    ratesVms_exdf.plot(x='Time/s', y=rate_cols, legend=None, ax=axs[0], color='gray', lw=0.5)
    ratesVms_exdf.plot(x='Time/s', y=Vm_cols, legend=None, ax=axs[1], color='gray', lw=0.5)
    axs[0].plot(ratesVms_exdf['Time/s'], ratesVms_exdf['Best Sample: Mean release rate/(ves./s)'], lw=1)
    axs[1].plot(ratesVms_exdf['Time/s'], ratesVms_exdf['Best Sample: Somatic membrane potential/mV'], lw=1)

# Plot.

## Parameters

In [ ]:
target_color = 'darkorange'
trace_color = 'C0'

target_peak_color = 'saddlebrown'

trace_peak_neg_color = 'c'
trace_peak_pos_color = 'navy'

## Plot helper functions

In [ ]:
def set_ylabel(label, ax, ylab_pad=5):
   
    ax.set_ylabel(
        label, rotation=90,
        horizontalalignment='center', verticalalignment='bottom',
        labelpad=ylab_pad
    )

In [ ]:
def plot_median_and_uncertainty(ax, time, data):
    
    assert np.all(np.isfinite(data))
    
    median = np.median(data, axis=0)
    lb = np.percentile(data, q=10, axis=0)
    ub = np.percentile(data, q=90, axis=0)
        
    ax.plot(time, median, c='k', linestyle='-', alpha=0.5, label='mean', clip_on=False, linewidth='0.5')
    ax.fill_between(time, lb, ub, alpha=0.3, facecolor="k", label='uncertainty', clip_on=False, zorder=10, lw=0)

In [ ]:
def plot_rest_hist(ax, data, bins):
    ax.hist(data, facecolor='gray', bins=bins, alpha=1, orientation='horizontal', lw=0)

### Plot stimulus

In [ ]:
stim_color = (0.8, 0.1, 0.1)

def plot_stim(ax, cell, c=stim_color, alpha=1):
    stim_df = cell2data[cell]['Stim']
    ax.plot(stim_df['Time'], stim_df['Stim'], c=c, alpha=alpha, lw=0.8)    
    set_ylabel(r'Light' + '\n' + '(norm.)', ax)

### Plot Vms

In [ ]:
def plot_Vm(ax, normalize=True, n_drop=0):
    
    rec_time = cell2data[cell]['rec_time']
    Vms = cell2data[cell]['Vms']*1e3
    Vm_best = cell2data[cell]['Vm_best']*1e3
    
    set_ylabel(r'V$_m$' + '\n' + '(mV)', ax)
    
    if normalize:
        Vm_best = Vm_best - Vm_best[n_drop]
        Vms = (Vms.T-Vms[:,n_drop]).T
        
    ax.plot(rec_time[n_drop:], Vm_best[n_drop:], c=trace_color, lw=0.8, clip_on=False)
    plot_median_and_uncertainty(ax, rec_time[n_drop:], Vms[:,n_drop:])

In [ ]:
def plot_Vm_rest(ax, cell, n_drop=0, bins=10):
    Vms = cell2data[cell]['Vms']*1e3
    plot_rest_hist(ax, data=Vms[:,n_drop], bins=bins)

### Plot rates.

In [ ]:
def plot_rate(ax, normalize=True, n_drop=0):
    
    rec_time = cell2data[cell]['rec_time']
    rates = cell2data[cell]['rates']
    rate_best = cell2data[cell]['rate_best']
    
    set_ylabel(r'Rate' + '\n' + '(ves./s)', ax)
    
    if normalize:
        rate_best = rate_best - rate_best[n_drop]
        rates = (rates.T-rates[:,n_drop]).T
    
    ax.plot(rec_time[n_drop:], rate_best[n_drop:], c=trace_color, lw=0.8, clip_on=False)
    plot_median_and_uncertainty(ax, rec_time[n_drop:], rates[:,n_drop:])

In [ ]:
def plot_rate_rest(ax, cell, n_drop=0, bins=10):
    rates = cell2data[cell]['rates']
    plot_rest_hist(ax, data=rates[:,n_drop], bins=bins)
    ax.set_ylim(0, None)

### Plot iGluSnFR traces

In [ ]:
n_xlims_ex = 3
xlims_ex = {}
xlims_ex['OFF'] = [(4.6, 5.6), (11.6, 14.1), (21, 25.5)]
xlims_ex['ON'] = [(1.6, 2.6), (11.6, 14.1), (21, 25.5)]
xlims_ex['Cone'] = [(1.6, 2.6), (11.5, 14), (23, 28.5)]

for xlims_ex_i in xlims_ex.values():
    assert len(xlims_ex_i) == n_xlims_ex

In [ ]:
def plot_iGlu(ax, cell, x_min_max=None, n_drop=0, plot_ylab=True):
   
    rec_time = cell2data[cell]['Time-Target']
    iGlus = cell2data[cell]['iGlus']
    iGlu_best = cell2data[cell]['iGlu_best']
    target = cell2data[cell]['Target']

    if plot_ylab: set_ylabel(r'iGluSnFR' + '\n' + '(norm.)', ax)
    
    if x_min_max is not None:
        idx = np.array((rec_time >= x_min_max[0]) & (rec_time <= x_min_max[1]))
        rec_time = rec_time[idx]
        target = target[idx]
        iGlu_best = iGlu_best[idx]
        iGlus = iGlus[:,idx]
    else:
        rec_time = rec_time[n_drop:]
        target = target[n_drop:]
        iGlu_best = iGlu_best[n_drop:]
        iGlus = iGlus[:,n_drop:]

    ax.plot(rec_time, target, c=target_color, alpha=1, lw=0.8)
    ax.plot(rec_time, iGlu_best, c=trace_color, alpha=1, lw=0.8)
    ax.plot(rec_time, target, c=target_color, alpha=0.4, lw=0.8)
    
    peak_iGlu_ylims = (np.min([np.min(target), np.min(iGlu_best)])-0.03,
                       np.max([np.max(target), np.max(iGlu_best)])+0.03)
    
    ax.set_ylim(peak_iGlu_ylims)
    
    if x_min_max is None:
        plot_median_and_uncertainty(ax, rec_time, iGlus)
        ax.set_yticks([0,1])
    else:
        ax.fill_between(rec_time, target, iGlu_best, color=target_color, alpha=0.3)

    return peak_iGlu_ylims

In [ ]:
def plot_rectangle(ax, x_min_max, y_min_max):
    ax.add_patch(
        Rectangle(
            (x_min_max[0], y_min_max[0]),
            x_min_max[1]-x_min_max[0],
            y_min_max[1]-y_min_max[0],
            alpha=1, linestyle='--', clip_on=False, lw=0.8, fill=False, edgecolor='gray'
        )
    )

### Plot peak histograms.

In [ ]:
summarized_peak_data = data_utils.load_var('preprocessed_data/summarized_peak_data.pkl')

In [ ]:
def plot_peak_hists(ax, cell):
    
    assert np.all(np.abs(summarized_peak_data[cell]['pos']['dt']) < 170)
    assert np.all(np.abs(summarized_peak_data[cell]['neg']['dt']) < 170)
    
    p1, p2, p3 = ax.hist(
        1000*np.array(summarized_peak_data[cell]['pos']['dt']),
        range=(-170,170), bins=20,
        alpha=0.6, facecolor=trace_peak_pos_color,
        edgecolor=None, density=False, linewidth=0
    )
    
    n1, n2, n3 = ax.hist(
        1000*np.array(summarized_peak_data[cell]['neg']['dt']),
        range=(-170,170), bins=20,
        alpha=0.6, color=trace_peak_neg_color,
        edgecolor=None, density=False, linewidth=0
    )
    
    hist_max = np.max([np.max(p1), np.max(n1)])
    
    ax.set_ylim(0,None)
    ax.set_ylabel('#peaks    ', labelpad=0)
    
    ax.plot([1000*np.median(summarized_peak_data[cell]['all']['dt'])]*2, [0, 1.1*hist_max], c='k')
    
    ax.set_xlabel(r'$\Delta t$ (ms)')
    
    yl = int(hist_max - (hist_max % 5))
    
    ticks = np.array([0, yl])
    ax.set_yticks(ticks)
    ax.set_yticklabels(ticks)
    ax.spines['left'].set_bounds(0, yl)

###### Export

In [ ]:
for cell in cell2data.keys():
    if cell == 'Cone':
        filename= 'source_data/Cone/Peak_data.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_Peak_data.csv'

    pd.DataFrame({
        'Target peak: Time/s': summarized_peak_data[cell]['all']['t0'],
        'Target amplitude: iGluSnFR': summarized_peak_data[cell]['all']['A0'],
        'Model peak: Time/s': summarized_peak_data[cell]['all']['t1'],
        'Model amplitude: iGluSnFR': summarized_peak_data[cell]['all']['A1'],
        'Difference: Time/ms': np.array(summarized_peak_data[cell]['all']['dt'])*1e3,
    }, index=['Positive' if t0 in summarized_peak_data[cell]['pos']['t0'] else 'Negative'\
              for t0 in summarized_peak_data[cell]['all']['t0']]).to_csv(filename, float_format="%.4f", index=True)

In [ ]:
fig, axs_all = plt.subplots(3,2,figsize=(12,5),gridspec_kw=dict(width_ratios=[3,1]), sharex='col')

for cell, axs in zip(cell2data.keys(), axs_all):
    if cell == 'Cone':
        filename= 'source_data/Cone/Peak_data.csv'
    else:
        filename= 'source_data/CBC/' + cell + '_Peak_data.csv'
        
    peak_exdf = pd.read_csv(filename, index_col=0)
    
    axs[0].plot(cell2data[cell]['Time-Target'], cell2data[cell]['Target'], c='b', alpha=0.5)
    axs[0].plot(cell2data[cell]['Time-Target'], cell2data[cell]['iGlu_best'], c='r', alpha=0.5)
    for idxs, marker in zip([peak_exdf.index == 'Positive', peak_exdf.index == 'Negative'], ['^', 'v']):
        axs[0].plot(peak_exdf['Target peak: Time/s'][idxs], peak_exdf['Target amplitude: iGluSnFR'][idxs],
                    marker, c='b', ms=5)
        axs[0].plot(peak_exdf['Model peak: Time/s'][idxs], peak_exdf['Model amplitude: iGluSnFR'][idxs],
                    marker, c='r', ms=5)
    
    for idxs, color in zip([peak_exdf.index == 'Positive', peak_exdf.index == 'Negative'], ['c', 'b']):
        axs[1].hist(peak_exdf['Difference: Time/ms'][idxs], range=(-170, 170), bins=21, alpha=0.3)

### Mark peaks in traces.

In [ ]:
def plot_peaks_func(ax, x_min_max, cell):
    ms = 4
    
    for pn in ['pos', 'neg']:
        if pn == 'pos':
            marker = '^'
            c0 = target_peak_color
            c1 = trace_peak_pos_color
        elif pn == 'neg':
            marker = 'v'
            c0 = target_peak_color
            c1 = trace_peak_neg_color
        
        # Plot connections.
        x0 = np.asarray(summarized_peak_data[cell][pn]['t0'])
        y0 = np.asarray(summarized_peak_data[cell][pn]['A0'])
        
        x1 = np.asarray(summarized_peak_data[cell][pn]['t1'])
        y1 = np.asarray(summarized_peak_data[cell][pn]['A1'])
        
        idxs = (x0 >= x_min_max[0]) & (x0 <= x_min_max[1]) & (x1 >= x_min_max[0]) & (x1 <= x_min_max[1])
        
        ax.plot([x0[idxs], x1[idxs]], [y0[idxs], y1[idxs]], '-', c='k', lw=0.6, zorder=120)
        
        ax.plot(x0[idxs], y0[idxs], marker, c=c0, markersize=ms, zorder=111, alpha=0.7, clip_on=False)
        ax.plot(x1[idxs], y1[idxs], marker, c=c1, markersize=ms, zorder=111, alpha=0.7, clip_on=False)

# Make figures

In [ ]:
abc = 'ABCDEFGHI'

In [ ]:
def plot_trace(ax, plot_type, cell, plotx=True):
    
    ax.set_xlim(0,32)
    
    if plot_type == 'Stim':
        plot_stim(ax, cell)
        ax.set_yticks([0,1])
    
    elif plot_type == 'Vms':
        plot_Vm(ax, cell)
        
    elif plot_type == 'rates':
        plot_rate(ax, cell)
        
    elif plot_type == 'iGlus':
        plot_iGlu(ax, cell)
        
    if plotx:
        ax.set_xlabel('Time (s)')
        ax.spines["bottom"].set_position(("axes", -0.1))
    else:
        ax.spines["bottom"].set_visible(False)
        ax.set_xticks([])

In [ ]:
def plot_hist(ax, plot_type, cell):
    
    if plot_type == 'Vms':
        plot_Vm_rest(ax, cell)
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.spines['bottom'].set_visible(False)
        
    elif plot_type == 'rates':
        plot_rate_rest(ax, cell)
        ax.set_xticks([])
        ax.set_xticklabels([])
        ax.spines['bottom'].set_visible(False)
        
    elif plot_type == 'iGlus':
        plot_peak_hists(ax, cell)
        ax.set_xlabel('Time (ms)')
        ax.spines["bottom"].set_position(("axes", -0.1))
        
    else:
        ax.axis('off')

## Cone

In [ ]:
plot_types = [('Cone', plot_type) for plot_type in ['Stim', 'Vms', 'rates', 'iGlus']]
y_frac = 0.75
y_gap = 0.05

fig = plt.figure(figsize=(7.9, (len(plot_types)+1)*0.65))

gs1 = fig.add_gridspec(4, 2, width_ratios=[5,1], height_ratios=[0.8, 0.8, 0.8, 1])
trace_axs = [fig.add_subplot(gs1[i,0]) for i in range(4)]
add_axs = [fig.add_subplot(gs1[i,1]) for i in range(4)]
gs1.tight_layout(fig, rect=[0.05, 1-y_frac, 1, 1], h_pad=-0.3)

gs2 = fig.add_gridspec(1, 3)
iGlu_axs = [fig.add_subplot(gs2[0,i]) for i in range(3)]
gs2.tight_layout(fig, rect=[0.05, -0.04, 1, 1-y_frac-y_gap], w_pad=-2, h_pad=0, pad=0)

sns.despine()
    
fig.align_ylabels(trace_axs)
    
for ax, add_ax, (cell, plot_type) in zip(trace_axs, add_axs, plot_types):
    plot_trace(ax, plot_type, cell, plotx=ax==trace_axs[-1])
    plot_hist(add_ax, plot_type, cell)
    
    if plot_type == 'iGlus':
        iGlu_main_ax = ax

    if plot_type == 'rates':
        ax.set_ylim(-50,35)
        
    if plot_type == 'Vms':
        ax.set_ylim(-6,0)

for ax, x_min_max in zip(iGlu_axs, xlims_ex['Cone']):
    ax.axis('off')
    y_min_max = plot_iGlu(ax, 'Cone', x_min_max=x_min_max, plot_ylab=False)
    plot_rectangle(ax, x_min_max, y_min_max)
    plot_rectangle(iGlu_main_ax, x_min_max, y_min_max)
    plot_peaks_func(ax, x_min_max, cell)
    
for ax, abc_i in zip(trace_axs, abc):
    ax.set_title(abc_i + '                ', loc='left', ha='right', va='top', fontweight="bold", pad=0)
    
plt.savefig(f'../_figures/fig{fig_num1}_cone_traces.pdf')

## CBC

In [ ]:
plot_types = [('OFF', plot_type) for plot_type in ['Stim', 'Vms', 'rates', 'iGlus']]
plot_types +=  [('ON', plot_type) for plot_type in ['Vms', 'rates', 'iGlus']]

height_ratios = [1, 0.0, 0.8,0.8,1,2, 0.8,0.8,1,2]

fig = plt.figure(figsize=(7.9, 7))

gs1 = fig.add_gridspec(len(height_ratios), 2, width_ratios=[5,1], height_ratios=height_ratios)
trace_axs = [fig.add_subplot(gs1[i,0]) for i, h in enumerate(height_ratios) if h in [0.8, 1]]
add_axs = [fig.add_subplot(gs1[i,1]) for i, h in enumerate(height_ratios) if h in [0.8, 1]]

fig.add_subplot(gs1[-1,0]).axis('off')
gs1.tight_layout(fig, rect=[0.05, 0.0, 1, 1], h_pad=-0.1)

sns.despine()

iGlu_main_axs = {}

for ax, add_ax, (cell, plot_type), abc_i in zip(trace_axs, add_axs, plot_types, abc):
    plot_trace(ax, plot_type, cell, plotx=plot_type=='iGlus')
    plot_hist(add_ax, plot_type, cell)
    ax.set_title(abc_i + '             ', loc='left', ha='right', va='center',fontweight="bold")

    if plot_type == 'rates':
        if cell == 'OFF':
            ax.set_ylim(-5,20)
        if cell == 'ON':
            ax.set_ylim(-5,30)
    
    if plot_type == 'Vms':
        if cell == 'OFF':
            ax.set_ylim(-15,10)
        if cell == 'ON':
            ax.set_ylim(-5,10)
    
    if plot_type == 'iGlus':
        ax.set_ylim(None, 1)
        iGlu_main_axs[cell] = ax
        
fig.align_ylabels(trace_axs)
        
for cell in ['OFF', 'ON']:

    gs = fig.add_gridspec(1, 3)
    iGlu_axs = [fig.add_subplot(gs[0,i]) for i in range(3)]
    
    y0 = -0.02 if cell == 'ON' else 0.42
        
    gs.tight_layout(fig, rect=[0.05, y0, 1, y0+0.15], w_pad=-2)
    
    for ax, x_min_max in zip(iGlu_axs, xlims_ex[cell]):
        ax.axis('off')
        y_min_max = plot_iGlu(ax, cell, x_min_max=x_min_max, plot_ylab=False)
        plot_rectangle(ax, x_min_max, y_min_max)
        plot_rectangle(iGlu_main_axs[cell], x_min_max, y_min_max)
        plot_peaks_func(ax, x_min_max, cell)

plt.savefig(f'../_figures/fig{fig_num2}_cbc_traces.pdf')

# Supplement figure

In [ ]:
ex_data_folder = os.path.join("..", "..", "data_experimental", "data_preprocessed")

drug_traces = data_utils.load_var(os.path.join(ex_data_folder, "drug_traces_sorted.pkl"))
cone_traces = data_utils.load_var(os.path.join(ex_data_folder, 'iGluSnFR_traces_Cones.pkl'))

##### Export

In [ ]:
cone_traces.to_csv(
    'source_data/Cone/iGluSnFR_experimental_recordings.csv', float_format="%.6f", index=False,
    header=['Time/s']+list(cone_traces.columns)[1:]
)

In [ ]:
cone_traces_exdf = pd.read_csv('source_data/Cone/iGluSnFR_experimental_recordings.csv')
cone_traces_exdf.plot(x='Time/s', y=['Cone1_mean_aligned', 'Cone2_mean_aligned']);

In [ ]:
for cell, cell_type in zip(['BC3a', 'BC5o'], ['OFF', 'ON']):
    n_traces = drug_traces['Strychnine'][cell]['traces'].shape[0]

    columns = ['Time/s'] + [cell+'_Strychnine: rec'+str(i+1) for i in range(n_traces)]

    cbc_traces_exdf = pd.DataFrame(columns=columns)

    cbc_traces_exdf[columns[0]] = drug_traces['Strychnine'][cell]['Time']
    cbc_traces_exdf[columns[1:]] = drug_traces['Strychnine'][cell]['traces'].T

    cbc_traces_exdf.to_csv(
        f'source_data/{cell_type}_iGluSnFR_experimental_recordings.csv',
        float_format="%.6f", index=False
    )

In [ ]:
for cell, cell_type in zip(['BC3a', 'BC5o'], ['OFF', 'ON']):
    cone_traces_exdf = pd.read_csv(f'source_data/{cell_type}_iGluSnFR_experimental_recordings.csv')
    cone_traces_exdf.plot(x='Time/s', figsize=(12,3), legend=False, c='k')

## Plot data

In [ ]:
def plot_single_traces(ax, time, traces, c='k'):
    ax.plot(time, traces, lw=0.1, clip_on=False, c=c)

In [ ]:
def plot_mean_and_std(ax, time, mean, std=None, c=target_color):
    ax.plot(time, mean, c=c, lw=0.8, clip_on=False)
    if std is not None:
        ax.fill_between(time, mean-std, mean+std,
                        color='dimgray', lw=0.1, alpha=0.4, clip_on=False)

In [ ]:
sbnx = 1
sbny = 6

fig, axs = plt.subplots(nrows=sbny, ncols=sbnx, figsize=(5.5,5.5), sharex=True)

cone1_cols = [col for col in cone_traces if 'Cone1_rec' in col]
cone2_cols = [col for col in cone_traces if 'Cone2_rec' in col]

normed_cone_traces = cone_traces[cone1_cols+cone2_cols].copy()
nct_max = cone_traces[cone1_cols+cone2_cols].max()
nct_min = cone_traces[cone1_cols+cone2_cols].min()
normed_cone_traces = (normed_cone_traces-nct_min)/(nct_max-nct_min)

plot_single_traces(axs[0], cone_traces['Time'], normed_cone_traces[cone1_cols], c='steelblue')
plot_single_traces(axs[0], cone_traces['Time'], normed_cone_traces[cone2_cols], c='k')

plot_mean_and_std(axs[1], cone_traces['Time'], cone_traces['Cone1_mean_aligned'], c='steelblue')
plot_mean_and_std(axs[1], cone_traces['Time'], cone_traces['Cone2_mean_aligned'], c='k')
plot_mean_and_std(axs[1], cone_traces['Time'], cone_traces['Cones_mean_of_means'])

for saxs, cell in zip([axs[2:4], axs[4:6]], ['BC3a', 'BC5o']):
    
    plot_single_traces(
        saxs[0],         
        drug_traces['Strychnine'][cell]['Time'],
        drug_traces['Strychnine'][cell]['traces'].T
    )
    
    plot_mean_and_std(
        saxs[1],
        drug_traces['Strychnine'][cell]['Time'],
        drug_traces['Strychnine'][cell]['mean'],
        drug_traces['Strychnine'][cell]['std'],
    )
    
plt.tight_layout(h_pad=1, pad=3)
    
axs[-1].set_xlabel('Time (s)')

for ax in axs: ax.set_ylabel(r'iGluSnFR' + '\n' + '(normalized)') 

for ax, a in zip(axs, "ABCDEF"):
    ax.set_yticks([0,1])
    ax.set_ylim(None, 1)
    ax.set_title(a + '           ', fontweight='bold', ha='right', loc='left', va='top')
    
sns.despine()

plt.savefig(f'../_figures_apx/figapx{fig_num1}_data_experimental.pdf')